# Assembly: conditionals
_COSC 208, Introduction to Computer Systems, 2024-03-29_

## Announcements
* Quiz 4 Wednesday
    * 2.4: Correct memory errors in C **code** with the assistance of a compiler and valgrind
    * 3.1: Design single- and multi-bit circuits for arithmetic and comparison operators
    * 3.2: Design a simple processing unit and explain how it executes assembly instructions
    * 4.1: Interpret ARM assembly code
    * 4.2: Determine the relationship between C code and ARM assembly code
    * Optional:
        * 2.1: Determine the contents of the stack and heap during a program’s execution
        * 2.2: Develop C **code** that appropriately uses arrays and pointer arithmetic

## Outline
* Warm-up
* Mapping C conditionals to assembly
* Transforming C code to goto-form
* Practice with conditionals

## Warm-up

* Q1: _When would the following function result in an error? How would you modify the code to avoid this error?_

In [ ]:
/*1*/  int divide(unsigned int numerator, unsigned int denominator) {
/*2*/      int result = -1;
/*3*/      result = numerator / denominator;
/*4*/      return result;
/*5*/  }

<p style="height:6em;"></p>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Mapping C conditionals to assembly

In [ ]:
/*1*/  int divide_safe(unsigned int numerator, unsigned int denominator) {
/*2*/      int result = -1;
/*3*/      if (denominator != 0) {
/*4*/          result = numerator / denominator;
/*5*/      }
/*6*/      return result;
/*7*/  }

```
000000000000076c <divide_safe>:                     // Line
    76c:    d10083ff     sub    sp, sp, #0x20       // 1
    770:    b9000fe0     str    w0, [sp, #12]       // 1
    774:    b9000be1     str    w1, [sp, #8]        // 1
    778:    12800000     mov    w0, #0xffffffff     // 2
    77c:    b9001fe0     str    w0, [sp, #28]       // 2
    780:    b9400be0     ldr    w0, [sp, #8]        // 3
    784:    7100001f     cmp    w0, #0x0            // 3
    788:    540000a0     b.eq   79c <divide_safe+0x30> // 3
    78c:    b9400fe1     ldr    w1, [sp, #12]       // 4
    790:    b9400be0     ldr    w0, [sp, #8]        // 4
    794:    1ac00c20     udiv   w0, w1, w0          // 4
    798:    b9001fe0     str    w0, [sp, #28]       // 4
    79c:    b9401fe0     ldr    w0, [sp, #28]       // 6
    7a0:    910083ff     add    sp, sp, #0x20       // 6
    7a4:    d65f03c0     ret                        // 6
```

```
000000000000076c <divide_safe>:
    76c:    d10083ff     sub    sp, sp, #0x20
    770:    b9000fe0     str    w0, [sp, #12]
    774:    b9000be1     str    w1, [sp, #8]
    778:    12800000     mov    w0, #0xffffffff
    77c:    b9001fe0     str    w0, [sp, #28]
    780:    b9400be0     ldr    w0, [sp, #8]
    784:    7100001f     cmp    w0, #0x0
    788:    540000a0     b.eq   79c <divide_safe+0x30>
    78c:    b9400fe1     ldr    w1, [sp, #12]
    790:    b9400be0     ldr    w0, [sp, #8]
    794:    1ac00c20     udiv   w0, w1, w0
    798:    b9001fe0     str    w0, [sp, #28]
    79c:    b9401fe0     ldr    w0, [sp, #28]
    7a0:    910083ff     add    sp, sp, #0x20
    7a4:    d65f03c0     ret
```

* _What does the `cmp` instruction do?_ — compares a register's value to another value
* _What does the `b.eq` instruction do?_ — "jumps" (i.e., branches) to a different instruction when the compared values are equal
* _Why does the assembly check if `w0 == 0` when the C code contains `!= 0`?_ — the C code checks for the condition that must be true to execute the if body, whereas the assembly code checks for the condition that must be true to **skip over** the if body

## Transforming C code to goto-form

* _How would we express this in C code?_ — using an if statement and a `goto` statement

In [ ]:
/*1*/  int divide_safe_goto(int numerator, int denominator) {
/*2*/      int result = -1;
/*3*/      if (denominator == 0)
/*4*/          goto after;
/*5*/      result = numerator / denominator;
/*6*/  after:
/*7*/      return result;
/*8*/  }

* _What happens if the code includes an else statement?_ — if condition is true, execute the if body and skip over the else body; if condition is false, skip over the if body and execute the else body

<div style="page-break-after:always;"></div>

## Practice with conditionals

* _The C code below was compiled into assembly (using `gcc`)._

In [ ]:
/* 1*/  int flip(int bit) {
/* 2*/     int result = -1;
/* 3*/     if (bit == 0) {
/* 4*/          result = 1; 
/* 5*/      } 
/* 6*/      else {
/* 7*/          result = 0;
/* 8*/      }
/* 9*/      return result;
/*10*/ }

* Q3: _For each line of assembly, indicate which original line of C code the assembly instruction was derived from._

```
000000000000071c <flip>:                            // Line
    71c:    d10083ff     sub    sp, sp, #0x20       // 1
    720:    b9000fe0     str    w0, [sp, #12]       // 1
    724:    12800000     mov    w0, #0xffffffff     // 2
    728:    b9001fe0     str    w0, [sp, #28]       // 2
    72c:    b9400fe0     ldr    w0, [sp, #12]       // 3
    730:    7100001f     cmp    w0, #0x0            // 3
    734:    54000081     b.ne   744 <flip+0x28>     // 3
    738:    52800020     mov    w0, #0x1            // 4
    73c:    b9001fe0     str    w0, [sp, #28]       // 4
    740:    14000002     b      748 <flip+0x2c>     // 5
    744:    b9001fff     str    wzr, [sp, #28]      // 7
    748:    b9401fe0     ldr    w0, [sp, #28]       // 9
    74c:    910083ff     add    sp, sp, #0x20       // 9
    750:    d65f03c0     ret                        // 9
```

```
000000000000071c <flip>:                            // Line
    71c:    d10083ff     sub    sp, sp, #0x20       // 
    720:    b9000fe0     str    w0, [sp, #12]       // 
    724:    12800000     mov    w0, #0xffffffff     // 
    728:    b9001fe0     str    w0, [sp, #28]       // 
    72c:    b9400fe0     ldr    w0, [sp, #12]       // 
    730:    7100001f     cmp    w0, #0x0            //
    734:    54000081     b.ne   744 <flip+0x28>     // 
    738:    52800020     mov    w0, #0x1            // 
    73c:    b9001fe0     str    w0, [sp, #28]       // 
    740:    14000002     b      748 <flip+0x2c>     // 
    744:    b9001fff     str    wzr, [sp, #28]      // 
    748:    b9401fe0     ldr    w0, [sp, #28]       // 
    74c:    910083ff     add    sp, sp, #0x20       // 
    750:    d65f03c0     ret                        // 
```

* Q4: _Write a function called `flip_goto` that behaves the same as `flip` but matches the structure of the assembly code that will be generated for `flip`. (Hint: you'll need two `goto` statements.)_

In [ ]:
int flip_goto(int bit) {
    int result = -1;
    if (bit != 0)
        goto else_body;
    result = 1; 
    goto after_else;
else_body:
    result = 0;
after_else:
    return result;
}

<p style="height:20em;"></p>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Mapping C loops to assembly

* Example mapping while loops to assembly

In [11]:
/* 1*/   #include <stdio.h>
/* 2*/   int pow2(int n) {
/* 3*/       int result = 1;
/* 4*/       while (n > 0) {
/* 5*/           result = result * 2;
/* 6*/           n = n - 1;
/* 7*/       }
/* 8*/       return result;
/* 9*/   }
/*10*/   int main() {
/*11*/    printf("2^10 = %d\n", pow2(10));
/*12*/}

2^10 = 1024


```
0000000000000754 <pow2>:
 754:   d10083ff        sub     sp, sp, #0x20    // 1
 758:   b9000fe0        str     w0, [sp, #12]    // 1
 75c:   52800020        mov     w0, #0x1         // 2
 760:   b9001fe0        str     w0, [sp, #28]    // 2
 764:   14000007        b       780 <pow2+0x2c>  // 3
 768:   b9401fe0        ldr     w0, [sp, #28]    // 4
 76c:   531f7800        lsl     w0, w0, #1       // 4
 770:   b9001fe0        str     w0, [sp, #28]    // 4
 774:   b9400fe0        ldr     w0, [sp, #12]    // 5
 778:   51000400        sub     w0, w0, #0x1     // 5
 77c:   b9000fe0        str     w0, [sp, #12]    // 5
 780:   b9400fe0        ldr     w0, [sp, #12]    // 3
 784:   7100001f        cmp     w0, #0x0         // 3
 788:   54ffff0c        b.gt    768 <pow2+0x14>  // 3
 78c:   b9401fe0        ldr     w0, [sp, #28]    // 7
 790:   910083ff        add     sp, sp, #0x20    // 7
 794:   d65f03c0        ret                      // 7
```

```
0000000000000754 <pow2>:
 754:   d10083ff        sub     sp, sp, #0x20
 758:   b9000fe0        str     w0, [sp, #12]
 75c:   52800020        mov     w0, #0x1
 760:   b9001fe0        str     w0, [sp, #28]
 764:   14000007        b       780 <pow2+0x2c>
 768:   b9401fe0        ldr     w0, [sp, #28]
 76c:   531f7800        lsl     w0, w0, #1
 770:   b9001fe0        str     w0, [sp, #28]
 774:   b9400fe0        ldr     w0, [sp, #12]
 778:   51000400        sub     w0, w0, #0x1
 77c:   b9000fe0        str     w0, [sp, #12]
 780:   b9400fe0        ldr     w0, [sp, #12]
 784:   7100001f        cmp     w0, #0x0
 788:   54ffff0c        b.gt    768 <pow2+0x14>
 78c:   b9401fe0        ldr     w0, [sp, #28]
 790:   910083ff        add     sp, sp, #0x20
 794:   d65f03c0        ret
```

<div style="page-break-after:always;"></div>

## Extra practice

* Q5: _Assume the registers currently hold the following values:_
```
sp = 0xA980
w/x10 = 10
w/x11 = 11
w/x12 = 12
w/x13 = 13
```
_Draw the contents of the stack and registers after the following instructions have been executed:_
```
sub sp, sp, #0x30
str w10, [sp, #16]
str x11, [sp]
str w12, [sp, #28]
str x13, [sp, #32]
ldr x10, [sp]
ldr w13, [sp, #28]
```

```
sp = 0xA950
w/x10 = 11
w/x11 = 11
w/x12 = 12
w/x13 = 12
```
```
        0  1  2  3  4  5  6  7  8          
 sp ->  +--+--+--+--+--+--+--+--+
0xA950  |          11           |
        +--+--+--+--+--+--+--+--+
0xA958  |           |           |
        +--+--+--+--+--+--+--+--+
0xA960  |    10     |           |
        +--+--+--+--+--+--+--+--+
0xA968  |           |     12    |
        +--+--+--+--+--+--+--+--+
0xA970  |          13           |
        +--+--+--+--+--+--+--+--+
0xA978  |                       |
        +--+--+--+--+--+--+--+--+
0xA980  |                       |
        +--+--+--+--+--+--+--+--+
```

<div style="height:15em;"></div>

* Q6: _The following C code was compiled into assembly (using `gcc`). Label each line of assembly code with the line number of the line of C code from which the assembly instruction was derived._ 

In [ ]:
/* 1*/   int abs(int value) {
/* 2*/       if (value < 0) {
/* 3*/           value = value * -1;
/* 4*/       }
/* 5*/       return value;
/* 6*/   }

```
000000000000074c <abs>:
    74c:   d10043ff        sub     sp, sp, #0x10    // 1
    750:   b9000fe0        str     w0, [sp, #12]    // 1
    754:   b9400fe0        ldr     w0, [sp, #12]    // 2
    758:   7100001f        cmp     w0, #0x0         // 2 
    75c:   5400008a        b.ge    76c <abs+0x20>   // 2
    760:   b9400fe0        ldr     w0, [sp, #12]    // 3
    764:   4b0003e0        neg     w0, w0           // 3
    768:   b9000fe0        str     w0, [sp, #12]    // 3
    76c:   b9400fe0        ldr     w0, [sp, #12]    // 5
    770:   910043ff        add     sp, sp, #0x10    // 5
    774:   d65f03c0        ret                      // 5
``` 

```
000000000000074c <abs>:
    74c:   d10043ff        sub     sp, sp, #0x10
    750:   b9000fe0        str     w0, [sp, #12]
    754:   b9400fe0        ldr     w0, [sp, #12]
    758:   7100001f        cmp     w0, #0x0 
    75c:   5400008a        b.ge    76c <abs+0x20>
    760:   b9400fe0        ldr     w0, [sp, #12]
    764:   4b0003e0        neg     w0, w0
    768:   b9000fe0        str     w0, [sp, #12]
    76c:   b9400fe0        ldr     w0, [sp, #12]
    770:   910043ff        add     sp, sp, #0x10
    774:   d65f03c0        ret
``` 

* Q7: _Write a function called `abs_goto` that behaves the same as `abs` but matches the structure of the assembly code that is generated for `abs`._

In [ ]:
int abs(int value) {
    if (value >= 0)
        goto after_if;
    value = value * -1;
after_if:
    return value;
}